### Script to convert geolocation into correct format

In [1]:
import geopandas as gpd
import pandas as pd
import os

In [3]:
%config Completer.use_jedi = False

### Settings

In [4]:
INFILE = 'nav.txt'
outfile = 'geo_pix4d.txt'
NEW_CRS = 'EPSG:4326'
H_ACC = 1
V_ACC = 1

#### Load images 

In [5]:
df = pd.read_csv(INFILE, sep='\t')

#### Change image suffixes 

In [6]:
images = df['File '].str.replace('.macs', '.tif')

In [7]:
images = images.apply(lambda x: x.strip().split('/')[-1])

#### Load coordinates to geom an reproject to selected crs 

In [8]:
#gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['latitude [decimal degrees]'], df['longitude [decimal degrees]']))

In [9]:
series = gpd.GeoSeries(gpd.points_from_xy(df['Lon[deg] '], df['Lat[deg] ']), crs='EPSG:4326')
series_new = series.to_crs(crs=NEW_CRS)

#### Fill Table 

In [10]:
df['imagename_tif'] = images
df['x'] = series_new.geometry.x
df['y'] = series_new.geometry.y
df['horizontal_accuracy'] = H_ACC
df['vertical_accuracy'] = V_ACC

#### Create final structure

In [11]:
df.columns

Index(['File ', 'Date ', 'Time ', 'msOfDay ', 'Lat[deg] ', 'Lon[deg] ',
       'Alt[m] ', 'Easting[m] ', 'Northing[m] ', 'Zone ', 'Roll[deg] ',
       'Pitch[deg] ', 'Yaw[deg] ', 'Omega[deg] ', 'Phi[deg] ', 'Kappa[deg]',
       'imagename_tif', 'x', 'y', 'horizontal_accuracy', 'vertical_accuracy'],
      dtype='object')

In [17]:
df_new = df[['imagename_tif', 'y', 'x', 'Alt[m] ', 'Omega[deg] ',
       'Phi[deg] ', 'Kappa[deg]', 'horizontal_accuracy', 'vertical_accuracy']]

In [18]:
df_new

,imagename_tif,y,x,Alt[m],Omega[deg],Phi[deg],Kappa[deg],horizontal_accuracy,vertical_accuracy
0,02268_004759891_1500.tif,68.731418,-133.588401,910.671063,-8.481340,-3.562697,132.369845,1,1
1,04564_010707893_1500.tif,68.705597,-133.697328,905.961692,-9.043587,-4.231675,129.822436,1,1
2,02267_004759391_1500.tif,68.731594,-133.587787,910.485519,-8.593521,-3.610940,132.184241,1,1
3,02269_004800391_1500.tif,68.731241,-133.589014,910.896608,-8.363080,-3.505972,132.406828,1,1
4,02266_004758891_1500.tif,68.731771,-133.587171,910.406976,-8.541458,-3.776054,131.920429,1,1
...,...,...,...,...,...,...,...,...,...
46212,18957_030706517_1800.tif,68.717368,-133.781215,892.885933,-0.742938,2.181139,130.806678,1,1
46213,18963_030709517_1800.tif,68.716344,-133.785046,892.711348,0.044048,2.691989,131.272922,1,1
46214,18965_030710517_1800.tif,68.715997,-133.786309,892.465831,-0.094110,2.670298,131.764798,1,1
46215,18959_030707517_1800.tif,68.717029,-133.782499,892.916433,-0.238094,2.582737,130.788446,1,1


#### Export file 

In [19]:
df_new.to_csv(outfile, sep='\t', header=True, index=False)